In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import pandas as pd


In [2]:
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_no_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_apply_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_combine_all_class_with_label_applied_fft.csv'
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_combine_all_class_with_label_no_fft.csv'
df = pd.read_csv(file_path)

In [3]:
# -- Prepare for modeling --
# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
#    - We'll drop 'start_time' because it's just a reference
#    - We'll keep 'label_encoded' as the target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,   # 20% for testing
    random_state=42  # for reproducibility
)

# Apply min-max normalization (0-1) to these columns
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# 4) Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5) Evaluate
y_pred = model.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred, target_names=le.classes_))

                   precision    recall  f1-score   support

  climbing_stairs       0.86      0.90      0.88        21
descending_stairs       0.96      0.79      0.87        33
          running       1.00      1.00      1.00        38
 sitting_standing       1.00      1.00      1.00        24
          walking       0.92      1.00      0.96        59

         accuracy                           0.95       175
        macro avg       0.95      0.94      0.94       175
     weighted avg       0.95      0.95      0.95       175

